In [1]:
## Parche de que si encuentro mas de una vez una seccion me quedo la primera y break
# PROTEINOGRAMA y PROTEINOGRAMA EN ORINA
# Hay un problema en peticion-160301-11063555.pdf y otros donde si bold (p.e. 14) está en otra línea cogerá esta
    # Troponina T cardiaca multiline se salta Creatin Quinasa

In [2]:
import re
import tabula
import pandas as pd
import fitz
import time
import math
import pdfplumber
import os
import time
import datetime

In [11]:
df = pd.read_excel('Sections.xlsx')
sections_excel = df['Sections'].tolist()
sections_excel

['BIOQUIMICA AUTOMATIZADA',
 'BIOQUÍMICA GENERAL',
 'IONOGRAMA',
 'CONTROL DE ANEMIA',
 'MARCADORES DE ISQUEMIA CARDIACA',
 'METABOLISMO LIPÍDICO',
 'MARCADORES DE ICC',
 'ENZIMAS',
 'PRUEBAS REUMÁTICAS',
 'NEFROLOGIA',
 'SISTEMÁTICO DE ORINA',
 'SEDIMENTO DE ORINA',
 'BIOQUÍMICA ORINA RECIENTE',
 'BIOQUIMICA ORINA 24H',
 'ACLARAMIENTO CREATININA',
 'ESTIMACIÓN FILTRADO GLOMERULAR',
 'HORMONAS',
 'TIROIDES',
 'MARCADORES TUMORALES',
 'BIOQUÍMICA GENERAL',
 'MARCADORES DE RIESGO CORONARIO',
 'PROTEINOGRAMA',
 'PROTEINOGRAMA EN ORINA',
 'PROTEINOGRAMA',
 'VITAMINAS',
 'HEMATIMETRÍA (sangre total)',
 'COAGULACIÓN',
 'BÁSICA (plasma citratado)',
 'BANCO DE SANGRE (INMUNOHEMATOLOGÍA)',
 'ERITROPATOLOGÍA',
 'CONTROL DE ANEMIA',
 'VELOCIDAD DE SEDIMENTACIÓN',
 'INMUNOQUÍMICA',
 'INMUNODEFICIENCIAS (S. INMUNOLOGÍA HCUVA)',
 'PRODUCCIÓN DE IFN GAMMA TB (MÉTODO QUANTIFERÓN)',
 'CITOMETRÍA - INMUNOFENOTIPO (S. INMUNOLOGÍA HCUVA)',
 'CITOMETRÍA - MOLECULAR (S. INMUNOLOGÍA HCUVA)',
 'GRUPO SANGUÍNE

In [4]:
ignore = [".", "blanco", "en blanco", "linea en blanco", "HEMATOLOGÍA", "Hemograma", "• Edad < 18 años", "Incidencias de muestras canceladas:", "INMUNOLOGIA", "Incidencias de pruebas:", "MICROBIOLOGÍA", "LABORATORIOS EXTERNOS"]

In [5]:
def remove_blanks(attributes, values):
    new_attributes = []
    new_values = []
    for idx, elem in enumerate(attributes):
        if elem == "": new_values[-1] += " " + values[idx]
        else:
            new_attributes.append(elem)
            new_values.append(values[idx])
    return new_attributes, new_values

In [6]:
def something(attributes):
    for elem in attributes:
        if elem == "": continue
        else: return True
    return False

In [7]:
def unique(list):
    unique_list = []
    for x in list:
        if x not in unique_list: unique_list.append(x)
    return unique_list

In [8]:
path = './Data/'
dirs = [x[0] for x in os.walk(path)]
dirs = dirs[1:]

#new = []

num_pages = 0
num_files = 0
num_values = 0

st = time.time()

for directory in dirs:
    files = os.listdir(directory)

    for file in files:
        num_files += 1
        path = directory+"/"+file
        print("ARCHIVO " + path)
        pdf = pdfplumber.open(path)
        doc = fitz.open(path)

        result = []

        for page in range(len(pdf.pages)):
            num_pages += 1
            print("PAGINA " + str(page+1))
            attributes_page = []
            values_page = []
            sections_coord = []
            sections_page = []

            '''
            # Leo la página y recojo en sections_page las secciones en azul del pdf
            lines = pdf.pages[page].filter(lambda obj: obj["object_type"] == "char" and "Bold" in obj["fontname"]).extract_text(x_tolerance=1)
            for line in lines.split('\n'):
                if line in sections: sections_page.append(line)
            '''
            lines = []
            text = pdf.pages[page].filter(lambda obj: obj["object_type"] == "char" and "Bold" in obj["fontname"]).extract_text(x_tolerance=1).split('\n')
            for line in text:
                if line not in ignore and not line.replace(".", '').isnumeric(): lines.append(line)
            for line in lines:
                coords = doc[page].search_for(line)
                if len(coords) > 1:
                    for coord in coords:
                        if coord.x0 < 29: 
                            if line in sections_excel: 
                                sections_coord.append((line, coord))
                                break
                            #else: new.append((line, file))
                if len(coords) > 0 and coords[0].x0 < 29: 
                    if line in sections_excel: sections_coord.append((line, coords[0]))
                    #else: new.append((line, file))
            sections_coord = unique(sections_coord)
            #print(sections_coord)
            if len(sections_coord) > 0: sections_page = list(zip(*sections_coord))[0]
            sections_page = unique(sections_page)
            #print(sections_page)

            '''
            # Busco las coordenadas de las secciones encontradas
            for section in sections_page:
                coords.append(doc[page].search_for(section)[0])
            '''

            # Si no hay secciones puede ser una ultima pagina, leer de principio a fin
            if len(sections_coord) == 0:
                area = (0, 110, pdf.pages[page].width, 787)
                # TODO Repeated code
                lines = pdf.pages[page].crop(area).extract_text(x_tolerance=1).split('\n')
                for line in lines:
                    if line not in ignore: attributes_page.append(line)
                bold = pdf.pages[page].crop(area).filter(lambda obj: obj["object_type"] == "char" and "Bold" in obj["fontname"]).extract_text(x_tolerance=1).split("\n")
                for line in bold: 
                    if line not in ignore and line not in sections_excel:
                        values_page.append(line)

            for idx, section_coord in enumerate(sections_coord):
                section = section_coord[0]
                coord = section_coord[1]
                
                # Si existe informacion previa a seccion
                if page != 0 and idx == 0 and coord.y0 > 135: 
                    area = (0, 110, pdf.pages[page].width, coord.y0)
                    # TODO Repeated code
                    lines = pdf.pages[page].crop(area).extract_text(x_tolerance=1).split('\n')
                    for line in lines:
                        if line not in ignore: attributes_page.append(line)
                    bold = pdf.pages[page].crop(area).filter(lambda obj: obj["object_type"] == "char" and "Bold" in obj["fontname"]).extract_text(x_tolerance=1).split("\n")
                    for line in bold: 
                        if line not in ignore and line not in sections_page:
                            if section == "HEMATIMETRÍA (sangre total)": line = line.lstrip("* ").split(" ") [0]
                            values_page.append(line)
                            
                if section == "PROTEINOGRAMA EN ORINA" or section == "PROTEINOGRAMA": continue
                
                # Ultima seccion
                if idx == len(sections_coord)-1: area = (0, coord.y1, pdf.pages[page].width, 787)
                # Seccion intermedia
                else: area = (0, coord.y1, pdf.pages[page].width, sections_coord[idx+1][1].y0)

                lines = pdf.pages[page].crop(area).extract_text(x_tolerance=1).split('\n')
                for line in lines:
                    if line not in ignore: attributes_page.append(line)
                bold = pdf.pages[page].crop(area).filter(lambda obj: obj["object_type"] == "char" and "Bold" in obj["fontname"]).extract_text(x_tolerance=1).split("\n")
                for line in bold: 
                    if line not in ignore and line not in sections_page:
                        if section == "HEMATIMETRÍA (sangre total)": line = line.lstrip("* ").split(" ") [0]
                        values_page.append(line)

            attributes_page = list(filter(None, attributes_page))
            values_page = list(filter(None, values_page))

            attributes_aux = attributes_page
            attributes_page = []
            n = 0
            for i in range(len(values_page)):
                for j in range(n, len(attributes_aux)):
                    if max(n, j) == j: n = j
                    coincide = False
                    if values_page[i] in attributes_aux[j]:
                        coincide = True
                        # Parche + coincide
                        if len(values_page[i].split(" ")) == 1:
                            coincide = False
                            for word in attributes_aux[j].split(" "):
                                if values_page[i] == word:
                                    coincide = True
                                    break
                        if coincide:
                            attributes_page.append(attributes_aux[j].split(values_page[i])[0].rstrip("* "))
                            n += 1
                            break   

            # Check si hay algun elemento en atributos que no sea empty y pueda añadirle el valor de values
            if "" in attributes_page:
                if something(attributes_page): attributes_page, values_page = remove_blanks(attributes_page, values_page)
                else: continue

            # Parche
            for idx, line in enumerate(attributes_page):
                if "Grupo sanguíneo" in line: attributes_page[idx] = "Grupo sanguíneo ABO"

            result = list(zip(attributes_page, values_page))
            for r in result:
                num_values += 1
                print(r)
            print()
            
            et = time.time()
            elapsed_time = et - st

ARCHIVO ./Data/120-5201356/peticion-150827-11801153.pdf
PAGINA 1
('Glucosa', '115')
('Urea', '32')
('Creatinina', '0.65')
('Acido úrico', '3.1')
('Proteínas totales', '6.7')
('Albúmina', '4.5')
('Calcio', '9.6')
('Fósforo', '4.0')
('Bilirrubina total', '0.44')
('Sodio', '141')
('Potasio', '4.2')
('Cloro', '98')
('Hierro', '174')
('Ferritina', '70')
('Transferrina', '242')
('Folato', '10.2')
('Vitamina B12', '223')
('Triglicéridos', '133')
('Colesterol', '236')
('GOT', '16')
('GPT', '16')
('GAMMA GT', '18')
('Fosfatasa alcalina', '46')
('LDH', '179')
('Proteína C Reactiva', '0.48')

PAGINA 2
('Densidad', '1.019')
('pH', '5.0')
('Albúmina', '25')
('Glucosa', 'Negativo')
('Cuerpos cetónicos', 'Negativo')
('Bilirrubina', 'Negativo')
('Hemoglobina', 'Positivo (+++)')
('Nitritos', 'Negativo')
('Urobilinógeno', '0.2')
('Leucocitos', 'Negativo')
('Leucocitos por campo', '3')
('Hematíes por campo', '5')
('Células escamosas', 'Algunas')
('Proteínas', '14')
('Creatinina', '52.7')
('MDRD', '98.49'

('Hematíes', '4.1')
('Hemoglobina', '11.6')
('Hematocrito', '35.8')
('Volumen corpuscular medio', '88.2')
('Hemoglobina corpuscular media', '28.6')
('Concentración hemoglobina corp. media', '32.4')
('Ancho de distribución eritrocitaria (CV)', '16.2')
('Eritroblastos', '0.00')
('Eritroblastos %', '0.00')
('Plaquetas', '324.0')
('Volumen plaquetario medio', '10.1')
('Ancho de distribución plaquetar', '11.5')
('Leucocitos', '9.60')
('Neutrófilos', '6.68')
('Linfocitos', '1.75')
('Monocitos', '0.79')
('Eosinófilos', '0.32')
('Basófilos', '0.06')
('Neutrófilos inmaduros', '0.030')
('Cociente granul. inm/ granul.t', '0.004')
('Tiempo de protrombina', '11.3')
('ratio Tiempo de Protrombina', '1.0')
('Actividad de protrombina', '93.0')
('Tiempo de tromboplastina parcial', '107.3')
('PTTA ratio', '3.43')
('INR', '1.04')
('Fibrinógeno derivado', '467')
('Tiempo de reptilase', '22.2')

PAGINA 5

ARCHIVO ./Data/127-730658/peticion-151126-11803924.pdf
PAGINA 1
('Glucosa', '90')
('Urea', '27')
('Crea

('Proteínas', '13')
('Proteínas 24h', '0.13')
('MDRD', '83.37')
('Ecuación CKD-EPI', '87.36')
('Beta 2-microglobulina', '2.96')

PAGINA 3
PAGINA 4
('Hematíes', '5.0')
('Hemoglobina', '13.1')
('Hematocrito', '42.3')
('Volumen corpuscular medio', '84.1')
('Hemoglobina corpuscular media', '26.0')
('Concentración hemoglobina corp. media', '31.0')
('Ancho de distribución eritrocitaria (CV)', '15.0')
('Hematíes hipocromos', '3.9')
('Reticulocitos', '44.30')
('Reticulocitos (%)', '0.88')
('Hemoglobina corpuscular reticulocitaria', '28.5')
('Eritroblastos', '0.00')
('Eritroblastos %', '0.00')
('Plaquetas', '192.0')
('Volumen plaquetario medio', '12.4')
('Ancho de distribución plaquetar', '17.2')
('Plaquetas reticuladas', '7.8')
('Leucocitos', '6.36')
('Neutrófilos', '3.44')
('Linfocitos', '1.90')
('Monocitos', '0.87')
('Eosinófilos', '0.12')
('Basófilos', '0.03')
('Neutrófilos inmaduros', '0.030')
('Cociente granul. inm/ granul.t', '0.009')
('Coombs directo', 'Negativo')
('Velocidad de sedimen

('Aclaramiento creatinina', '77')
('MDRD', '93.89')
('Ecuación CKD-EPI', '92.54')
('T4 libre', '1.28')
('TSH', '0.352')
('Beta 2-microglobulina', '1.35')
('Homocisteína', '14.3')

PAGINA 4
('ALFA-1 CALCULADA', '0.2')
('ALFA-1 GLOBULINA', '4.3')
('ALFA-2 CALCULADA', '0.7')
('ALFA-2 GLOBULINA', '12.1')
('BETA-1 CALCULADA', '0.4')
('BETA-1 GLOBULINA', '6.2')
('BETA-2 CALCULADA', '0.2')
('BETA-2 GLOBULINA', '3.7')
('GAMMA CALCULADA', '0.4')
('GAMMA GLOBULINA', '7.0')
('COCIENTE A/G', '2.01')

PAGINA 5
('Hematíes', '3.4')
('Hemoglobina', '11.4')
('Hematocrito', '34.3')
('Volumen corpuscular medio', '100.0')
('Hemoglobina corpuscular media', '33.2')
('Concentración hemoglobina corp. media', '33.2')
('Ancho de distribución eritrocitaria (CV)', '17.2')
('Hematíes hipocromos', '0.3')
('Reticulocitos', '48.40')
('Reticulocitos (%)', '1.41')
('Hemoglobina corpuscular reticulocitaria', '33.5')
('Eritroblastos', '0.00')
('Eritroblastos %', '0.00')
('Plaquetas', '279.0')
('Volumen plaquetario medio'

('Glucosa', '97')
('Urea', '33')
('Creatinina', '0.78')
('Acido úrico', '4.5')
('Proteínas totales', '6.4')
('Albúmina', '4.2')
('Calcio', '9.2')
('Fósforo', '2.9')
('Magnesio', '1.90')
('Bilirrubina total', '0.31')
('Sodio', '143')
('Potasio', '4.1')
('Cloro', '105')
('Ferritina', '18')
('Transferrina', '282')
('Troponina T cardíaca', '<3')
('Triglicéridos', '134')
('Colesterol', '218')

PAGINA 2
('GOT (AST)', '18')
('GPT (ALT)', '12')
('GAMMA GT', '17')
('Fosfatasa alcalina', '50')
('Creatin Quinasa', '21')
('LDH', '159')
('Proteína C Reactiva', '0.52')
('Proteínas', '5')
('MDRD', '77.60')
('Ecuación CKD-EPI', '77.27')
('Beta 2-microglobulina', '2.40')

PAGINA 3
('Homocisteína', '10.7')

PAGINA 4
('Hematíes', '4.7')
('Hemoglobina', '12.5')
('Hematocrito', '39.9')
('Volumen corpuscular medio', '85.1')
('Hemoglobina corpuscular media', '26.7')
('Concentración hemoglobina corp. media', '31.3')
('Ancho de distribución eritrocitaria (CV)', '17.4')
('Hematíes hipocromos', '2.2')
('Reticulo

...

('CPs aberrantes del total de CPs (%)', '0.90')
('CD38+++IgG citop+ (%)', '0.07')
('CD38+++Kappa citop+ (%)', '0.07')
('CD38+++Lambda citop+ (%)', '0.04')
('CD38+++IgGcit+Lcit+ (%)', '0.03')
('CD38+++IgGcit+Kcit+ (%)', '0.05')
('CD43+ (%)', '96.26')
('CD56+CD3- (%)', '0.9170')
('CD117+ (%)', '5.84')
('Linfocitos (%)', '0.02')

ARCHIVO ./Data/200-418416/peticion-210628-12140589.pdf
PAGINA 1
('Glucosa', '93')
('Urea', '43')
('Creatinina', '1.31')
('Acido úrico', '4.8')
('Proteínas totales', '6.3')
('Albúmina', '4.6')
('Calcio', '9.6')
('Fósforo', '3.6')
('Magnesio', '2.13')
('Bilirrubina total', '0.41')
('Sodio', '142')
('Potasio', '4.1')
('Cloro', '103')
('Hierro', '86')
('Ferritina', '170')
('Transferrina', '257')
('Indice saturación transferrina', '23.3')
('Folato', '4.2')
('Vitamina B12', '1529')

PAGINA 2
('Troponina T cardíaca', '7')
('Triglicéridos', '109')
('Colesterol', '248')
('HDL-colesterol', '55')
('LDL-colesterol (calculado)', '172')
('Colesterol no HDL (calculado)', '194')

('Glucosa', '90')
('Urea', '85')
('Creatinina', '1.98')
('Acido úrico', '10.1')
('Proteínas totales', '6.3')
('Albúmina', '4.4')
('Calcio', '9.2')
('Fósforo', '3.9')
('Magnesio', '2.14')
('Bilirrubina total', '0.21')
('Sodio', '138')
('Potasio', '4.4')
('Cloro', '98')
('Hierro', '55')
('Ferritina', '82')
('Transferrina', '235')
('Indice saturación transferrina', '16.3')
('Folato', '4.2')
('Vitamina B12', '418')

PAGINA 2
('Troponina T cardíaca', '9')
('Triglicéridos', '270')
('Niveles recomendados inferiores a', '150')
('NT-proBNP', '66')
('GOT (AST)', '15')
('GPT (ALT)', '19')
('GAMMA GT', '14')
('Fosfatasa alcalina', '75')
('Creatin Quinasa', '44')
('LDH', '228')
('Proteína C Reactiva', '0.49')

PAGINA 3
('Densidad', '1.014')
('pH', '6.0')
('Glucosa', 'Normal')
('Bilirrubina', 'Negativo')
('Cuerpos cetónicos', 'Negativo')
('Urobilinógeno', 'Normal')
('Proteínas', 'Negativo')
('Nitritos', 'Negativo')
('Leucocitos', 'Positivo (+++)')
('Sangre', 'Positivo (+)')
('Leucocitos por campo', 

PAGINA 1
('Glucosa', '84')
('Urea', '19')
('Creatinina', '0.43')
('Acido úrico', '2.7')
('Proteínas totales', '5.7')
('Albúmina', '4.5')
('Calcio', '8.8')
('Fósforo', '2.7')
('Magnesio', '1.82')
('Bilirrubina total', '0.98')
('Sodio', '144')
('Potasio', '4.5')
('Cloro', '108')
('Hierro', '132')
('Ferritina', '230')
('Transferrina', '235')
('Indice saturación transferrina', '39.3')
('Folato', '12.3')
('Vitamina B12', '896')

PAGINA 2
('Troponina T cardíaca', '<3')
('Triglicéridos', '189')
('Colesterol', '210')
('GOT (AST)', '20')
('GPT (ALT)', '37')
('GAMMA GT', '20')
('Fosfatasa alcalina', '73')
('Creatin Quinasa', '27')
('LDH', '158')
('Proteína C Reactiva', '0.14')
('Densidad', '1.016')
('pH', '7.0')
('Glucosa', 'Normal')
('Bilirrubina', 'Negativo')
('Cuerpos cetónicos', 'Negativo')
('Urobilinógeno', 'Normal')
('Proteínas', 'Negativo')
('Nitritos', 'Negativo')
('Leucocitos', 'Negativo')
('Sangre', 'Negativo')

PAGINA 3
('Creatinina', '37.7')
('Proteínas', '9')
('Proteínas 24h', '0.24

('Glucosa', '247')
('Urea', '61')
('Creatinina', '3.52')
('Acido úrico', '4.2')
('Proteínas totales', '6.6')
('Albúmina', '3.8')
('Calcio', '9.3')
('Fósforo', '3.9')
('Bilirrubina total', '0.42')
('Sodio', '139')
('Potasio', '3.7')
('Cloro', '100')
('Triglicéridos', '286')
('Colesterol', '181')
('GPT (ALT)', '9')
('GAMMA GT', '9')
('Fosfatasa alcalina', '99')

PAGINA 2
('Proteína C Reactiva', '0.49')
('MDRD', '19.09')
('Ecuación CKD-EPI', '18.04')

PAGINA 3
('Hematíes', '4.2')
('Hemoglobina', '11.7')
('Hematocrito', '35.2')
('Volumen corpuscular medio', '82.8')
('Hemoglobina corpuscular media', '27.5')
('Concentración hemoglobina corp. media', '33.2')
('Ancho de distribución eritrocitaria (CV)', '17.9')
('Hematíes hipocromos', '3.8')
('Reticulocitos', '68.00')
('Reticulocitos (%)', '1.60')
('Hemoglobina corpuscular reticulocitaria', '28.6')
('Eritroblastos', '0.00')
('Eritroblastos %', '0.00')
('Plaquetas', '207.0')
('Volumen plaquetario medio', '10.9')
('Ancho de distribución plaqueta

('Glucosa', '152')
('Urea', '67')
('Creatinina', '3.81')
('Acido úrico', '5.9')
('Albúmina', '3.9')
('Calcio', '9.6')
('Calcio sérico (corregido por albumina)', '9.70')
('Fósforo', '4.0')
('Sodio', '140')
('Potasio', '4.9')
('Hierro', '80')
('Ferritina', '122')
('Transferrina', '226')
('Indice saturación transferrina', '24.9')
('GOT (AST)', '6')
('GPT (ALT)', '12')

PAGINA 2
('Ecuación CKD-EPI', '11.6')
('PTH basal', '136.0')

PAGINA 3
('Hematíes', '2.7')
('Hemoglobina', '10.1')
('Hematocrito', '28.9')
('Volumen corpuscular medio', '107.4')
('Hemoglobina corpuscular media', '37.5')
('Concentración hemoglobina corp. media', '34.9')
('Ancho de distribución eritrocitaria (CV)', '14.6')
('Ancho de distribución eritrocitaria (DS)', '56.8')
('Eritroblastos', '0.00')
('Eritroblastos %', '0.00')
('Plaquetas', '179.0')
('Volumen plaquetario medio', '10.2')
('Leucocitos', '4.15')
('Neutrófilos', '3.71')
('Linfocitos', '0.36')
('Monocitos', '0.07')
('Eosinófilos', '0.00')
('Basófilos', '0.01')
('

In [9]:
print("Files Pages Attributes Time")
print(num_files, num_pages, num_values, str(datetime.timedelta(seconds=elapsed_time)))

Files Pages Attributes Time
134 650 8090 0:01:14.808033
